# 第七章：型態、方法與介面

本章將深入探討 Go 的型態系統、方法定義以及介面的使用。這些是 Go 語言中實現抽象和模組化程式設計的核心概念。

## 1. Go 中的型態

在 Go 中，我們可以定義自己的型態。這些型態可以基於內建型態，也可以是結構體。定義自己的型態有助於：

- **型態安全**: 防止混淆不同概念的相同底層型態
- **程式碼可讀性**: 讓程式碼表達更清楚的意圖
- **方法附加**: 可以為自定義型態添加方法

### 基本型態定義

使用 `type` 關鍵字來定義新型態：

**型態別名 vs 新型態**: Go 區分型態別名 (`type NewType = OldType`) 和新型態定義 (`type NewType OldType`)。

**底層型態**: 新定義的型態與其底層型態在記憶體表示上相同，但在型態檢查上是不同的。

**型態轉換**: 不同的型態即使底層相同也不能直接賦值，需要明確轉換。

**方法附加**: 只有新定義的型態（非型態別名）才能添加方法。

簡單範例：
```go
type UserID int       // 新型態，可以添加方法
type Age = int        // 型態別名，不能添加方法

var id UserID = 123
var normalInt int = int(id)  // 需要轉換
```

In [ ]:
package main

import "fmt"

// 定義基於內建型態的新型態
type Score int        // 分數型態
type UserID string    // 使用者 ID 型態
type Temperature float64  // 溫度型態

func main() {
    // 使用自定義型態
    var gameScore Score = 85
    var userID UserID = "user123"
    var temp Temperature = 25.5
    
    fmt.Printf("Game Score: %d\n", gameScore)
    fmt.Printf("User ID: %s\n", userID)
    fmt.Printf("Temperature: %.1f°C\n", temp)
    
    // 型態安全：不能直接賦值不同型態，即使底層型態相同
    var normalInt int = 100
    // gameScore = normalInt  // 這會編譯錯誤！
    gameScore = Score(normalInt)  // 需要明確轉換
    fmt.Printf("Converted Score: %d\n", gameScore)
}

### 結構體型態定義

結構體是更複雜的自定義型態：

**結構體特性**: 結構體將相關的資料欄位組織在一起，是 Go 中建立自定義資料型態的主要方式。

**初始化方式**: 支援多種初始化語法，包括具名初始化、位置初始化和零值初始化。

**記憶體對齊**: Go 會自動處理欄位的記憶體對齊，但欄位順序可能影響記憶體使用量。

**可見性規則**: 大寫開頭的欄位和型態名稱可以跨套件存取，小寫則只能在同套件內使用。

**嵌入支援**: 結構體支援匿名欄位嵌入，實現組合式的程式設計。

範例：
```go
type Person struct {
    Name  string    // 公開欄位
    age   int       // 私有欄位
    Email string
}

// 建立實例
p1 := Person{Name: "Alice", Email: "alice@example.com"}
p2 := Person{"Bob", 25, "bob@example.com"}  // 按順序初始化
```

In [ ]:
package main

import "fmt"

// 定義結構體型態
type Person struct {
    Name string
    Age  int
    Email string
}

type Rectangle struct {
    Width  float64
    Height float64
}

func main() {
    // 建立結構體實例
    p1 := Person{
        Name:  "Alice",
        Age:   30,
        Email: "alice@example.com",
    }
    
    // 另一種建立方式
    p2 := Person{"Bob", 25, "bob@example.com"}
    
    // 零值建立
    var p3 Person  // 所有欄位都是零值
    p3.Name = "Charlie"
    p3.Age = 35
    
    fmt.Printf("Person 1: %+v\n", p1)
    fmt.Printf("Person 2: %+v\n", p2)
    fmt.Printf("Person 3: %+v\n", p3)
    
    rect := Rectangle{Width: 10.5, Height: 8.2}
    fmt.Printf("Rectangle: %+v\n", rect)
}

## 2. 方法

Go 支援為自定義型態定義方法。方法是與特定型態關聯的函式。

### 基本方法定義

方法的語法格式：`func (接收器) 方法名(參數) 回傳值`

**接收器 (Receiver)**: 方法與特定型態關聯的機制，可以是值接收器或指標接收器。

**方法 vs 函數**: 方法有接收器，屬於特定型態；函數沒有接收器，是獨立的。

**方法集 (Method Set)**: 型態所有方法的集合，決定了型態可以實現哪些介面。

**呼叫語法**: 使用點記號 `receiver.Method()` 呼叫方法，Go 會自動處理值和指標之間的轉換。

**命名約定**: 接收器變數通常使用型態名稱的第一個字母（小寫）。

基本範例：
```go
type Rectangle struct {
    Width, Height float64
}

// 值接收器方法
func (r Rectangle) Area() float64 {
    return r.Width * r.Height
}

// 指標接收器方法
func (r *Rectangle) Scale(factor float64) {
    r.Width *= factor
    r.Height *= factor
}
```

In [ ]:
package main

import (
    "fmt"
    "strings"
)

type Person struct {
    Name string
    Age  int
}

type Rectangle struct {
    Width  float64
    Height float64
}

// 為 Person 定義方法
func (p Person) String() string {
    return fmt.Sprintf("Person{Name: %s, Age: %d}", p.Name, p.Age)
}

func (p Person) IsAdult() bool {
    return p.Age >= 18
}

func (p Person) Greet() string {
    return fmt.Sprintf("Hello, I'm %s", p.Name)
}

// 為 Rectangle 定義方法
func (r Rectangle) Area() float64 {
    return r.Width * r.Height
}

func (r Rectangle) Perimeter() float64 {
    return 2 * (r.Width + r.Height)
}

func (r Rectangle) String() string {
    return fmt.Sprintf("Rectangle{Width: %.2f, Height: %.2f}", r.Width, r.Height)
}

func main() {
    // 使用 Person 方法
    p := Person{Name: "Alice", Age: 25}
    fmt.Println(p.String())       // 呼叫 String 方法
    fmt.Println(p.Greet())        // 呼叫 Greet 方法
    fmt.Printf("Is adult: %t\n", p.IsAdult())  // 呼叫 IsAdult 方法
    
    // 使用 Rectangle 方法
    rect := Rectangle{Width: 10.5, Height: 8.0}
    fmt.Println(rect.String())
    fmt.Printf("Area: %.2f\n", rect.Area())
    fmt.Printf("Perimeter: %.2f\n", rect.Perimeter())
}

### 為內建型態定義方法

我們也可以為基於內建型態的自定義型態定義方法：

In [ ]:
package main

import (
    "fmt"
    "strings"
)

// 基於內建型態的自定義型態
type MyString string
type Counter int
type StringSlice []string

// 為 MyString 定義方法
func (ms MyString) Upper() MyString {
    return MyString(strings.ToUpper(string(ms)))
}

func (ms MyString) Lower() MyString {
    return MyString(strings.ToLower(string(ms)))
}

func (ms MyString) Reverse() MyString {
    runes := []rune(string(ms))
    for i, j := 0, len(runes)-1; i < j; i, j = i+1, j-1 {
        runes[i], runes[j] = runes[j], runes[i]
    }
    return MyString(string(runes))
}

// 為 Counter 定義方法
func (c *Counter) Increment() {
    *c++  // 注意這裡使用指標接收器來修改值
}

func (c *Counter) Decrement() {
    *c--
}

func (c Counter) Value() int {
    return int(c)
}

// 為 StringSlice 定義方法
func (ss StringSlice) Join(separator string) string {
    return strings.Join(ss, separator)
}

func (ss StringSlice) Contains(item string) bool {
    for _, s := range ss {
        if s == item {
            return true
        }
    }
    return false
}

func main() {
    // 使用 MyString 方法
    text := MyString("Hello World")
    fmt.Printf("Original: %s\n", text)
    fmt.Printf("Upper: %s\n", text.Upper())
    fmt.Printf("Lower: %s\n", text.Lower())
    fmt.Printf("Reverse: %s\n", text.Reverse())
    
    // 使用 Counter 方法
    var counter Counter
    fmt.Printf("Initial counter: %d\n", counter.Value())
    counter.Increment()
    counter.Increment()
    fmt.Printf("After increment: %d\n", counter.Value())
    counter.Decrement()
    fmt.Printf("After decrement: %d\n", counter.Value())
    
    // 使用 StringSlice 方法
    fruits := StringSlice{"apple", "banana", "orange"}
    fmt.Printf("Fruits: %s\n", fruits.Join(", "))
    fmt.Printf("Contains 'banana': %t\n", fruits.Contains("banana"))
    fmt.Printf("Contains 'grape': %t\n", fruits.Contains("grape"))
}

## 3. 指標接收器與值接收器

Go 的方法接收器可以是值接收器或指標接收器。選擇哪一種很重要：

### 值接收器 vs 指標接收器

- **值接收器**: 方法接收到接收器的副本，無法修改原始值
- **指標接收器**: 方法接收到接收器的指標，可以修改原始值

In [ ]:
package main

import "fmt"

type BankAccount struct {
    Owner   string
    Balance float64
}

// 值接收器 - 不會修改原始結構體
func (ba BankAccount) GetBalance() float64 {
    return ba.Balance  // 讀取操作，使用值接收器
}

func (ba BankAccount) String() string {
    return fmt.Sprintf("Account{Owner: %s, Balance: %.2f}", ba.Owner, ba.Balance)
}

// 指標接收器 - 會修改原始結構體
func (ba *BankAccount) Deposit(amount float64) {
    if amount > 0 {
        ba.Balance += amount  // 修改操作，使用指標接收器
        fmt.Printf("Deposited %.2f, new balance: %.2f\n", amount, ba.Balance)
    }
}

func (ba *BankAccount) Withdraw(amount float64) bool {
    if amount > 0 && amount <= ba.Balance {
        ba.Balance -= amount  // 修改操作，使用指標接收器
        fmt.Printf("Withdrew %.2f, new balance: %.2f\n", amount, ba.Balance)
        return true
    }
    fmt.Printf("Cannot withdraw %.2f, insufficient funds\n", amount)
    return false
}

func (ba *BankAccount) SetOwner(newOwner string) {
    ba.Owner = newOwner  // 修改操作，使用指標接收器
}

func main() {
    // 建立銀行帳戶
    account := BankAccount{
        Owner:   "Alice",
        Balance: 1000.0,
    }
    
    fmt.Println("=== 初始狀態 ===")
    fmt.Println(account.String())
    fmt.Printf("Balance: %.2f\n", account.GetBalance())
    
    fmt.Println("\n=== 存款操作 ===")
    account.Deposit(500.0)  // Go 自動處理 &account
    fmt.Println(account.String())
    
    fmt.Println("\n=== 提款操作 ===")
    account.Withdraw(200.0)
    account.Withdraw(2000.0)  // 餘額不足
    fmt.Println(account.String())
    
    fmt.Println("\n=== 修改擁有者 ===")
    account.SetOwner("Bob")
    fmt.Println(account.String())
    
    // 展示指標和值的差異
    fmt.Println("\n=== 指標 vs 值的差異 ===")
    account2 := BankAccount{Owner: "Charlie", Balance: 100.0}
    
    // 透過值呼叫（內部會自動轉換成指標）
    account2.Deposit(50.0)
    
    // 透過指標明確呼叫
    accountPtr := &account2
    accountPtr.Withdraw(30.0)
    
    fmt.Println(account2.String())
}

### 選擇接收器型態的準則

選擇指標接收器的情況：
1. 方法需要修改接收器
2. 接收器是大型結構體（避免複製開銷）
3. 為了一致性（如果某個方法使用指標接收器，其他方法也應該使用）

In [ ]:
package main

import (
    "fmt"
    "time"
)

// 大型結構體範例
type LargeStruct struct {
    Data [1000]int  // 大量資料
    Name string
    ID   int
}

// 小型結構體範例
type Point struct {
    X, Y float64
}

// 大型結構體使用指標接收器（避免複製開銷）
func (ls *LargeStruct) ProcessData() {
    // 處理大量資料，使用指標接收器避免複製
    for i := range ls.Data {
        ls.Data[i] = i * 2
    }
}

func (ls *LargeStruct) GetSummary() string {
    // 即使是讀取操作，大型結構體也建議使用指標接收器
    return fmt.Sprintf("LargeStruct{Name: %s, ID: %d, DataSize: %d}", 
                      ls.Name, ls.ID, len(ls.Data))
}

// 小型結構體可以使用值接收器
func (p Point) Distance() float64 {
    // 小型結構體的讀取操作，可以使用值接收器
    return p.X*p.X + p.Y*p.Y  // 平方距離
}

func (p *Point) Move(dx, dy float64) {
    // 修改操作必須使用指標接收器
    p.X += dx
    p.Y += dy
}

func (p Point) String() string {
    // 小型結構體的字串表示，使用值接收器
    return fmt.Sprintf("Point{X: %.2f, Y: %.2f}", p.X, p.Y)
}

func main() {
    // 大型結構體範例
    large := &LargeStruct{
        Name: "BigData",
        ID:   1001,
    }
    
    fmt.Println("=== 大型結構體操作 ===")
    fmt.Println(large.GetSummary())
    
    start := time.Now()
    large.ProcessData()
    fmt.Printf("Processing time: %v\n", time.Since(start))
    
    // 小型結構體範例
    fmt.Println("\n=== 小型結構體操作 ===")
    point := Point{X: 3.0, Y: 4.0}
    fmt.Println(point.String())
    fmt.Printf("Square distance: %.2f\n", point.Distance())
    
    point.Move(1.0, -1.0)
    fmt.Printf("After move: %s\n", point.String())
    
    // 性能比較示例（概念展示）
    fmt.Println("\n=== 性能考慮 ===")
    fmt.Println("大型結構體應該使用指標接收器避免複製開銷")
    fmt.Println("小型結構體可以根據是否需要修改來選擇接收器類型")
}

## 4. 程式碼重用與嵌入

Go 支援結構體嵌入，這是一種組合（composition）的方式來重用程式碼。

### 結構體嵌入基礎

In [ ]:
package main

import "fmt"

// 基礎型態
type Person struct {
    Name string
    Age  int
}

type Address struct {
    Street string
    City   string
    ZipCode string
}

// 為基礎型態定義方法
func (p Person) Introduce() string {
    return fmt.Sprintf("Hi, I'm %s, %d years old", p.Name, p.Age)
}

func (a Address) FullAddress() string {
    return fmt.Sprintf("%s, %s %s", a.Street, a.City, a.ZipCode)
}

// 使用嵌入的複合型態
type Employee struct {
    Person    // 嵌入 Person
    Address   // 嵌入 Address
    JobTitle  string
    Salary    float64
    EmployeeID int
}

type Student struct {
    Person     // 嵌入 Person
    StudentID  string
    Major      string
    GPA        float64
}

// 為複合型態定義專屬方法
func (e Employee) GetSalaryInfo() string {
    return fmt.Sprintf("Employee %d earns $%.2f as %s", 
                      e.EmployeeID, e.Salary, e.JobTitle)
}

func (s Student) GetAcademicInfo() string {
    return fmt.Sprintf("Student %s studies %s with GPA %.2f", 
                      s.StudentID, s.Major, s.GPA)
}

func main() {
    // 建立員工
    emp := Employee{
        Person: Person{
            Name: "Alice",
            Age:  30,
        },
        Address: Address{
            Street:  "123 Main St",
            City:    "New York",
            ZipCode: "10001",
        },
        JobTitle:   "Software Engineer",
        Salary:     85000.0,
        EmployeeID: 1001,
    }
    
    // 建立學生
    student := Student{
        Person: Person{
            Name: "Bob",
            Age:  22,
        },
        StudentID: "S123456",
        Major:     "Computer Science",
        GPA:       3.8,
    }
    
    fmt.Println("=== 員工資訊 ===")
    // 可以直接呼叫嵌入型態的方法
    fmt.Println(emp.Introduce())        // Person 的方法
    fmt.Println(emp.FullAddress())      // Address 的方法
    fmt.Println(emp.GetSalaryInfo())    // Employee 自己的方法
    
    // 可以直接存取嵌入型態的欄位
    fmt.Printf("Employee Name: %s\n", emp.Name)  // Person.Name
    fmt.Printf("Employee City: %s\n", emp.City)  // Address.City
    
    fmt.Println("\n=== 學生資訊 ===")
    fmt.Println(student.Introduce())       // Person 的方法
    fmt.Println(student.GetAcademicInfo()) // Student 自己的方法
    fmt.Printf("Student Age: %d\n", student.Age)  // Person.Age
    
    // 也可以明確指定嵌入型態
    fmt.Println("\n=== 明確存取嵌入型態 ===")
    fmt.Printf("Employee Person: %+v\n", emp.Person)
    fmt.Printf("Employee Address: %+v\n", emp.Address)
}

### 方法重寫與嵌入

嵌入型態可以重寫嵌入的方法：

In [ ]:
package main

import "fmt"

type Animal struct {
    Name string
    Species string
}

func (a Animal) Speak() string {
    return fmt.Sprintf("%s makes a generic animal sound", a.Name)
}

func (a Animal) Info() string {
    return fmt.Sprintf("%s is a %s", a.Name, a.Species)
}

// Dog 嵌入 Animal
type Dog struct {
    Animal  // 嵌入
    Breed   string
}

// Dog 重寫 Speak 方法
func (d Dog) Speak() string {
    return fmt.Sprintf("%s barks: Woof! Woof!", d.Name)
}

// Dog 添加新方法
func (d Dog) Fetch() string {
    return fmt.Sprintf("%s is fetching the ball!", d.Name)
}

// Cat 嵌入 Animal
type Cat struct {
    Animal    // 嵌入
    IndoorOnly bool
}

// Cat 重寫 Speak 方法
func (c Cat) Speak() string {
    return fmt.Sprintf("%s meows: Meow! Meow!", c.Name)
}

// Cat 重寫 Info 方法並調用嵌入型態的方法
func (c Cat) Info() string {
    baseInfo := c.Animal.Info()  // 明確呼叫嵌入型態的方法
    if c.IndoorOnly {
        return baseInfo + " (indoor only)"
    }
    return baseInfo + " (outdoor allowed)"
}

func main() {
    // 建立動物實例
    genericAnimal := Animal{
        Name:    "Unknown",
        Species: "Generic Animal",
    }
    
    dog := Dog{
        Animal: Animal{
            Name:    "Buddy",
            Species: "Canine",
        },
        Breed: "Golden Retriever",
    }
    
    cat := Cat{
        Animal: Animal{
            Name:    "Whiskers",
            Species: "Feline",
        },
        IndoorOnly: true,
    }
    
    fmt.Println("=== 一般動物 ===")
    fmt.Println(genericAnimal.Speak())
    fmt.Println(genericAnimal.Info())
    
    fmt.Println("\n=== 狗 ===")
    fmt.Println(dog.Speak())      // 使用重寫的方法
    fmt.Println(dog.Info())       // 使用嵌入的方法
    fmt.Println(dog.Fetch())      // 使用 Dog 特有的方法
    
    // 也可以存取嵌入型態的原始方法
    fmt.Println("Original animal sound:", dog.Animal.Speak())
    
    fmt.Println("\n=== 貓 ===")
    fmt.Println(cat.Speak())      // 使用重寫的方法
    fmt.Println(cat.Info())       // 使用重寫的方法（呼叫了嵌入型態的方法）
    
    fmt.Println("\n=== 嵌入型態存取 ===")
    fmt.Printf("Dog breed: %s\n", dog.Breed)
    fmt.Printf("Dog name (via embedding): %s\n", dog.Name)
    fmt.Printf("Cat indoor only: %t\n", cat.IndoorOnly)
    fmt.Printf("Cat species (via embedding): %s\n", cat.Species)
}

## 5. 嵌入不是繼承

重要提醒：Go 的嵌入是組合（composition），不是繼承（inheritance）。這有一些重要的含義：

In [ ]:
package main

import "fmt"

type Base struct {
    Name string
}

func (b Base) Method1() string {
    return fmt.Sprintf("Base.Method1 called by %s", b.Name)
}

func (b Base) Method2() string {
    // 這個方法呼叫同一個接收器的另一個方法
    return fmt.Sprintf("Base.Method2 calls Method1: %s", b.Method1())
}

type Derived struct {
    Base  // 嵌入
    Extra string
}

// Derived 重寫 Method1
func (d Derived) Method1() string {
    return fmt.Sprintf("Derived.Method1 called by %s (extra: %s)", d.Name, d.Extra)
}

func main() {
    base := Base{Name: "BaseInstance"}
    derived := Derived{
        Base:  Base{Name: "DerivedInstance"},
        Extra: "additional info",
    }
    
    fmt.Println("=== Base 實例 ===")
    fmt.Println(base.Method1())
    fmt.Println(base.Method2())
    
    fmt.Println("\n=== Derived 實例 ===")
    fmt.Println(derived.Method1())  // 呼叫重寫的方法
    fmt.Println(derived.Method2())  // 嵌入的方法，但它內部呼叫的 Method1 仍是 Base 的版本！
    
    fmt.Println("\n=== 重要觀察 ===")
    fmt.Println("當 Base.Method2 呼叫 Method1 時，它呼叫的是 Base.Method1，")
    fmt.Println("而不是 Derived.Method1。這證明了嵌入不是繼承！")
    
    fmt.Println("\n=== 型態轉換測試 ===")
    // 嵌入不支援多型
    var baseVar Base
    // baseVar = derived  // 這會編譯錯誤！
    baseVar = derived.Base  // 必須明確取得嵌入的部分
    fmt.Printf("baseVar from derived.Base: %s\n", baseVar.Method1())
    
    // 展示組合的本質
    fmt.Println("\n=== 組合的本質 ===")
    fmt.Printf("derived.Base: %+v\n", derived.Base)
    fmt.Printf("derived 本身包含 Base: %+v\n", derived)
    
    // 可以獨立修改嵌入的部分
    derived.Base.Name = "ModifiedBase"
    fmt.Printf("修改後的 derived.Name: %s\n", derived.Name)
    fmt.Printf("修改後的 derived.Base.Name: %s\n", derived.Base.Name)
}

## 6. 介面（Interface）

Go 的介面是隱式的，不需要明確宣告某個型態實現了某個介面。只要型態實現了介面中的所有方法，就自動實現了該介面。

### 介面基礎

In [ ]:
package main

import (
    "fmt"
    "math"
)

// 定義介面
type Shape interface {
    Area() float64        // 計算面積
    Perimeter() float64   // 計算周長
}

type Drawable interface {
    Draw() string        // 繪製
}

// 複合介面
type DrawableShape interface {
    Shape           // 嵌入 Shape 介面
    Drawable        // 嵌入 Drawable 介面
}

// 實現型態
type Rectangle struct {
    Width  float64
    Height float64
}

type Circle struct {
    Radius float64
}

type Triangle struct {
    Base   float64
    Height float64
    Side1  float64
    Side2  float64
}

// Rectangle 實現 Shape 介面
func (r Rectangle) Area() float64 {
    return r.Width * r.Height
}

func (r Rectangle) Perimeter() float64 {
    return 2 * (r.Width + r.Height)
}

func (r Rectangle) Draw() string {
    return fmt.Sprintf("Drawing a rectangle %.1f x %.1f", r.Width, r.Height)
}

// Circle 實現 Shape 介面
func (c Circle) Area() float64 {
    return math.Pi * c.Radius * c.Radius
}

func (c Circle) Perimeter() float64 {
    return 2 * math.Pi * c.Radius
}

func (c Circle) Draw() string {
    return fmt.Sprintf("Drawing a circle with radius %.1f", c.Radius)
}

// Triangle 實現 Shape 介面
func (t Triangle) Area() float64 {
    return 0.5 * t.Base * t.Height
}

func (t Triangle) Perimeter() float64 {
    return t.Base + t.Side1 + t.Side2
}

func (t Triangle) Draw() string {
    return fmt.Sprintf("Drawing a triangle with base %.1f and height %.1f", t.Base, t.Height)
}

// 使用介面的函式
func PrintShapeInfo(s Shape) {
    fmt.Printf("Shape - Area: %.2f, Perimeter: %.2f\n", s.Area(), s.Perimeter())
}

func DrawShape(d Drawable) {
    fmt.Println(d.Draw())
}

func ProcessDrawableShape(ds DrawableShape) {
    fmt.Println(ds.Draw())
    PrintShapeInfo(ds)  // DrawableShape 也實現了 Shape
}

func main() {
    // 建立不同的形狀
    rect := Rectangle{Width: 10, Height: 5}
    circle := Circle{Radius: 3}
    triangle := Triangle{Base: 6, Height: 4, Side1: 5, Side2: 5}
    
    // 將具體型態當作介面使用
    shapes := []Shape{rect, circle, triangle}
    
    fmt.Println("=== 使用 Shape 介面 ===")
    for i, shape := range shapes {
        fmt.Printf("Shape %d: ", i+1)
        PrintShapeInfo(shape)
    }
    
    fmt.Println("\n=== 使用 Drawable 介面 ===")
    drawables := []Drawable{rect, circle, triangle}
    for _, drawable := range drawables {
        DrawShape(drawable)
    }
    
    fmt.Println("\n=== 使用複合介面 DrawableShape ===")
    drawableShapes := []DrawableShape{rect, circle, triangle}
    for _, ds := range drawableShapes {
        fmt.Println("---")
        ProcessDrawableShape(ds)
    }
    
    // 介面變數
    fmt.Println("\n=== 介面變數 ===")
    var s Shape
    s = rect
    fmt.Printf("Rectangle as Shape: Area = %.2f\n", s.Area())
    
    s = circle
    fmt.Printf("Circle as Shape: Area = %.2f\n", s.Area())
}

## 7. 介面與 nil

Go 中的介面值包含兩個部分：具體型態和具體值。理解介面與 nil 的關係很重要：

In [ ]:
package main

import "fmt"

type Writer interface {
    Write(data string) error
}

type FileWriter struct {
    filename string
}

func (fw *FileWriter) Write(data string) error {
    if fw == nil {
        return fmt.Errorf("FileWriter is nil")
    }
    fmt.Printf("Writing '%s' to file: %s\n", data, fw.filename)
    return nil
}

type NullWriter struct{}

func (nw NullWriter) Write(data string) error {
    fmt.Printf("NullWriter: discarding '%s'\n", data)
    return nil
}

func processWriter(w Writer) {
    fmt.Printf("Writer is nil: %t\n", w == nil)
    if w != nil {
        err := w.Write("test data")
        if err != nil {
            fmt.Printf("Error: %v\n", err)
        }
    } else {
        fmt.Println("Writer is nil, cannot write")
    }
}

func main() {
    fmt.Println("=== 介面與 nil 的各種情況 ===")
    
    // 情況 1: nil 介面
    fmt.Println("\n1. nil 介面:")
    var w1 Writer
    fmt.Printf("w1 == nil: %t\n", w1 == nil)
    processWriter(w1)
    
    // 情況 2: 非 nil 介面，具體值為 nil
    fmt.Println("\n2. 非 nil 介面，具體值為 nil:")
    var fw *FileWriter  // fw 是 nil 指標
    var w2 Writer = fw  // 介面不是 nil，但具體值是 nil
    fmt.Printf("fw == nil: %t\n", fw == nil)
    fmt.Printf("w2 == nil: %t\n", w2 == nil)  // false！
    processWriter(w2)  // 會呼叫 Write 方法，但在方法內檢查 nil
    
    // 情況 3: 正常的非 nil 介面和值
    fmt.Println("\n3. 正常的非 nil 介面和值:")
    fw2 := &FileWriter{filename: "output.txt"}
    var w3 Writer = fw2
    fmt.Printf("w3 == nil: %t\n", w3 == nil)
    processWriter(w3)
    
    // 情況 4: 值型態實現介面
    fmt.Println("\n4. 值型態實現介面:")
    nw := NullWriter{}
    var w4 Writer = nw
    fmt.Printf("w4 == nil: %t\n", w4 == nil)
    processWriter(w4)
    
    // 展示介面內部結構
    fmt.Println("\n=== 介面內部結構展示 ===")
    fmt.Printf("nil 介面: %v\n", w1)
    fmt.Printf("具體值為 nil 的介面: %v\n", w2)
    fmt.Printf("正常介面: %v\n", w3)
    fmt.Printf("值型態介面: %v\n", w4)
    
    // 安全的 nil 檢查方法
    fmt.Println("\n=== 安全的 nil 檢查 ===")
    interfaces := []Writer{w1, w2, w3, w4}
    for i, w := range interfaces {
        fmt.Printf("Interface %d: ", i+1)
        if w == nil {
            fmt.Println("is nil interface")
        } else {
            fmt.Printf("is not nil interface, value: %v\n", w)
            // 對於可能包含 nil 指標的介面，在實現中檢查
            w.Write("safe check data")
        }
    }
}

## 8. 空介面與型態斷言

Go 的空介面 `interface{}` 可以持有任何型態的值。配合型態斷言，可以實現更靈活的程式設計：

In [ ]:
package main

import "fmt"

// 處理任意型態的函式
func ProcessAny(value interface{}) {
    fmt.Printf("Processing value: %v (type: %T)\n", value, value)
    
    // 型態斷言
    if str, ok := value.(string); ok {
        fmt.Printf("  String value length: %d\n", len(str))
    } else if num, ok := value.(int); ok {
        fmt.Printf("  Integer value doubled: %d\n", num*2)
    } else if slice, ok := value.([]int); ok {
        sum := 0
        for _, v := range slice {
            sum += v
        }
        fmt.Printf("  Slice sum: %d\n", sum)
    } else {
        fmt.Printf("  Unknown type: %T\n", value)
    }
}

// 使用型態切換的函式
func ProcessWithSwitch(value interface{}) {
    switch v := value.(type) {
    case string:
        fmt.Printf("String: '%s' (length: %d)\n", v, len(v))
    case int:
        fmt.Printf("Int: %d (is even: %t)\n", v, v%2 == 0)
    case int64:
        fmt.Printf("Int64: %d\n", v)
    case float64:
        fmt.Printf("Float64: %.2f\n", v)
    case bool:
        fmt.Printf("Bool: %t\n", v)
    case []int:
        fmt.Printf("Int slice: %v (length: %d)\n", v, len(v))
    case map[string]int:
        fmt.Printf("String-Int map: %v\n", v)
    case nil:
        fmt.Println("Nil value")
    default:
        fmt.Printf("Unknown type: %T, value: %v\n", v, v)
    }
}

// 自定義型態
type Person struct {
    Name string
    Age  int
}

func (p Person) String() string {
    return fmt.Sprintf("Person{Name: %s, Age: %d}", p.Name, p.Age)
}

// 型態斷言錯誤示例
func UnsafeTypeAssertion(value interface{}) {
    defer func() {
        if r := recover(); r != nil {
            fmt.Printf("Recovered from panic: %v\n", r)
        }
    }()
    
    // 危險的型態斷言（沒有檢查）
    str := value.(string)  // 如果不是 string 會 panic
    fmt.Printf("String: %s\n", str)
}

func main() {
    // 準備測試資料
    values := []interface{}{
        "Hello, World!",
        42,
        int64(1000),
        3.14159,
        true,
        []int{1, 2, 3, 4, 5},
        map[string]int{"apple": 5, "banana": 3},
        Person{Name: "Alice", Age: 30},
        nil,
    }
    
    fmt.Println("=== 使用型態斷言 ===")
    for _, value := range values {
        ProcessAny(value)
        fmt.Println()
    }
    
    fmt.Println("=== 使用型態切換 ===")
    for _, value := range values {
        ProcessWithSwitch(value)
    }
    
    // 展示安全和不安全的型態斷言
    fmt.Println("\n=== 安全 vs 不安全的型態斷言 ===")
    
    var testValue interface{} = 42
    
    // 安全的方式
    if str, ok := testValue.(string); ok {
        fmt.Printf("Safely got string: %s\n", str)
    } else {
        fmt.Printf("Value is not a string (it's %T)\n", testValue)
    }
    
    // 不安全的方式（會 panic）
    fmt.Println("\nTrying unsafe type assertion:")
    UnsafeTypeAssertion(testValue)  // 會被 recover 捕捉
    
    // 展示空介面切片
    fmt.Println("\n=== 空介面切片 ===")
    mixed := []interface{}{"string", 123, true, []int{1, 2, 3}}
    for i, item := range mixed {
        fmt.Printf("Item %d: %v (%T)\n", i, item, item)
    }
}

## 9. 函式型態作為介面

在 Go 中，函式型態也可以實現介面，這提供了一種優雅的方式來創建簡單的實現：

In [ ]:
package main

import (
    "fmt"
    "sort"
    "strconv"
    "strings"
)

// Handler 介面
type Handler interface {
    Handle(request string) string
}

// 函式型態實現 Handler 介面
type HandlerFunc func(string) string

func (hf HandlerFunc) Handle(request string) string {
    return hf(request)  // 呼叫函式本身
}

// Processor 介面
type Processor interface {
    Process(data []string) []string
}

// 函式型態實現 Processor 介面
type ProcessorFunc func([]string) []string

func (pf ProcessorFunc) Process(data []string) []string {
    return pf(data)
}

// 具體的結構體實現
type EchoHandler struct {
    prefix string
}

func (eh EchoHandler) Handle(request string) string {
    return fmt.Sprintf("%s: %s", eh.prefix, request)
}

// 使用 Handler 的函式
func ProcessRequest(h Handler, request string) {
    result := h.Handle(request)
    fmt.Printf("Request: %s -> Response: %s\n", request, result)
}

// 使用 Processor 的函式
func RunProcessor(p Processor, data []string) {
    fmt.Printf("Input: %v\n", data)
    result := p.Process(data)
    fmt.Printf("Output: %v\n", result)
    fmt.Println()
}

func main() {
    fmt.Println("=== 函式型態作為介面實現 ===")
    
    // 1. 使用結構體實現
    echo := EchoHandler{prefix: "ECHO"}
    ProcessRequest(echo, "hello world")
    
    // 2. 使用函式型態實現 - 直接定義函式
    upperHandler := HandlerFunc(func(request string) string {
        return strings.ToUpper(request)
    })
    ProcessRequest(upperHandler, "hello world")
    
    // 3. 使用已定義的函式
    ProcessRequest(HandlerFunc(reverseString), "hello world")
    
    // 4. 函式型態處理器的實際應用
    fmt.Println("\n=== 資料處理器範例 ===")
    
    data := []string{"apple", "Banana", "cherry", "Date"}
    
    // 轉大寫處理器
    upperProcessor := ProcessorFunc(func(data []string) []string {
        result := make([]string, len(data))
        for i, s := range data {
            result[i] = strings.ToUpper(s)
        }
        return result
    })
    
    // 排序處理器
    sortProcessor := ProcessorFunc(func(data []string) []string {
        result := make([]string, len(data))
        copy(result, data)
        sort.Strings(result)
        return result
    })
    
    // 過濾處理器（只保留長度 > 5 的字串）
    filterProcessor := ProcessorFunc(func(data []string) []string {
        var result []string
        for _, s := range data {
            if len(s) > 5 {
                result = append(result, s)
            }
        }
        return result
    })
    
    // 添加索引處理器
    indexProcessor := ProcessorFunc(func(data []string) []string {
        result := make([]string, len(data))
        for i, s := range data {
            result[i] = strconv.Itoa(i+1) + ". " + s
        }
        return result
    })
    
    // 使用不同的處理器
    processors := []Processor{
        upperProcessor,
        sortProcessor,
        filterProcessor,
        indexProcessor,
    }
    
    names := []string{"Upper", "Sort", "Filter", "Index"}
    
    for i, processor := range processors {
        fmt.Printf("--- %s Processor ---\n", names[i])
        RunProcessor(processor, data)
    }
    
    // 鏈式處理
    fmt.Println("=== 鏈式處理 ===")
    result := data
    fmt.Printf("Original: %v\n", result)
    
    // 依序套用處理器
    result = upperProcessor.Process(result)
    fmt.Printf("After upper: %v\n", result)
    
    result = sortProcessor.Process(result)
    fmt.Printf("After sort: %v\n", result)
    
    result = indexProcessor.Process(result)
    fmt.Printf("After index: %v\n", result)
}

// 輔助函式
func reverseString(s string) string {
    runes := []rune(s)
    for i, j := 0, len(runes)-1; i < j; i, j = i+1, j-1 {
        runes[i], runes[j] = runes[j], runes[i]
    }
    return string(runes)
}

## 10. 介面的最佳實踐

### 接受介面，回傳具體型態

這是 Go 的一個重要設計原則：

In [ ]:
package main

import (
    "fmt"
    "io"
    "strings"
)

// 小介面的例子
type Reader interface {
    Read() (string, error)
}

type Writer interface {
    Write(data string) error
}

// 具體實現
type FileManager struct {
    content []string
    index   int
}

func NewFileManager() *FileManager {
    return &FileManager{
        content: []string{"line 1", "line 2", "line 3"},
        index:   0,
    }
}

func (fm *FileManager) Read() (string, error) {
    if fm.index >= len(fm.content) {
        return "", io.EOF
    }
    line := fm.content[fm.index]
    fm.index++
    return line, nil
}

func (fm *FileManager) Write(data string) error {
    fm.content = append(fm.content, data)
    return nil
}

func (fm *FileManager) Reset() {
    fm.index = 0
}

func (fm *FileManager) GetAllContent() []string {
    return fm.content
}

// 好的設計：接受介面，回傳具體型態
func ProcessAndCreateManager(r Reader) *FileManager {
    manager := NewFileManager()
    
    for {
        line, err := r.Read()
        if err == io.EOF {
            break
        }
        if err != nil {
            fmt.Printf("Error reading: %v\n", err)
            break
        }
        
        // 處理資料並寫入新管理器
        processed := fmt.Sprintf("Processed: %s", strings.ToUpper(line))
        manager.Write(processed)
    }
    
    return manager  // 回傳具體型態
}

func CopyData(src Reader, dst Writer) error {
    for {
        data, err := src.Read()
        if err == io.EOF {
            break
        }
        if err != nil {
            return err
        }
        
        if err := dst.Write(data); err != nil {
            return err
        }
    }
    return nil
}

// 小介面組合
type ReadWriter interface {
    Reader
    Writer
}

func ProcessReadWriter(rw ReadWriter) {
    fmt.Println("=== 處理 ReadWriter ===")
    
    // 先讀取所有內容
    var lines []string
    for {
        line, err := rw.Read()
        if err == io.EOF {
            break
        }
        if err != nil {
            fmt.Printf("Error: %v\n", err)
            return
        }
        lines = append(lines, line)
    }
    
    fmt.Printf("Read %d lines\n", len(lines))
    
    // 寫入處理後的內容
    for i, line := range lines {
        processed := fmt.Sprintf("[%d] %s", i+1, line)
        rw.Write(processed)
    }
}

// 介面隔離原則的示例
type Printer interface {
    Print() string
}

func (fm *FileManager) Print() string {
    return fmt.Sprintf("FileManager with %d lines", len(fm.content))
}

// 只需要 Print 功能的函式
func DisplayInfo(p Printer) {
    fmt.Printf("Display: %s\n", p.Print())
}

func main() {
    fmt.Println("=== 介面最佳實踐示範 ===")
    
    // 建立原始資料
    source := NewFileManager()
    
    fmt.Println("\n1. 接受介面，回傳具體型態:")
    // ProcessAndCreateManager 接受 Reader 介面，回傳具體的 *FileManager
    processed := ProcessAndCreateManager(source)
    
    fmt.Printf("Original content: %v\n", source.GetAllContent())
    fmt.Printf("Processed content: %v\n", processed.GetAllContent())
    
    // 因為回傳具體型態，我們可以使用所有方法
    processed.Reset()
    fmt.Println("Reset processed manager")
    
    fmt.Println("\n2. 小介面組合:")
    target := NewFileManager()
    
    // 重置 source 以便重新讀取
    source.Reset()
    err := CopyData(source, target)
    if err != nil {
        fmt.Printf("Copy error: %v\n", err)
    }
    
    fmt.Printf("Target after copy: %v\n", target.GetAllContent())
    
    fmt.Println("\n3. 介面組合使用:")
    manager := NewFileManager()
    manager.Reset()  // 確保從頭開始讀
    ProcessReadWriter(manager)  // FileManager 實現了 ReadWriter
    
    fmt.Printf("Manager after processing: %v\n", manager.GetAllContent())
    
    fmt.Println("\n4. 介面隔離原則:")
    DisplayInfo(manager)  // 只使用 Printer 介面
    
    fmt.Println("\n=== 設計原則總結 ===")
    fmt.Println("✓ 接受介面：讓函式更靈活，可以接受任何實現該介面的型態")
    fmt.Println("✓ 回傳具體型態：讓呼叫者可以存取完整的 API")
    fmt.Println("✓ 小介面：更容易實現和組合")
    fmt.Println("✓ 介面隔離：只依賴需要的方法")
}

## 總結

本章涵蓋了 Go 語言中型態、方法與介面的核心概念：

### 重點回顧

1. **自定義型態**：提供型態安全和程式碼可讀性
2. **方法**：為型態添加行為，注意值接收器與指標接收器的選擇
3. **嵌入**：透過組合而非繼承來重用程式碼
4. **介面**：隱式實現，提供靈活的抽象
5. **介面與 nil**：理解介面值的結構很重要
6. **型態斷言**：安全地處理空介面
7. **函式型態**：可以實現介面，提供優雅的解決方案

### 最佳實踐

- 接受介面，回傳具體型態
- 保持介面小而專注
- 使用組合而非繼承
- 在需要修改或大型結構體時使用指標接收器
- 進行安全的型態斷言

### 注意事項

- 嵌入不是繼承，理解其限制
- 介面包含 nil 指標時不等於 nil
- 型態斷言失敗會 panic，要使用安全形式
- 方法集合規則影響介面實現